In [43]:
import pandas as pd
import numpy as np

In [44]:
import re, collections

In [45]:
from os.path import dirname, join

In [46]:
def clean_up (file_name):
    with open(file_name) as infile:
        contents = infile.read()
        # replacements
        rep = {'%20': ' ','%2F%2F':'://', '%2F':'/',';':'\n','.png':'.png \n','%3A':''}
        for i, j in rep.iteritems():
            contents = contents.replace(i, j)
    return contents
        

In [47]:
data = clean_up('./Test_results/alexa200_data.txt')

In [48]:
def prepare(file_name, from_file=False):
    collection = []
    data = collections.defaultdict(dict)
    count_num_data = 0
    
    # checking for expernal or internal file source
    if from_file:
        with open(file_name, 'r') as l_file:
            content = l_file.read()
    else:
        content = file_name
    
    # separate each param and add to list
    line = content.split('\n')
    
    for each in line:
        
        values = each.split(':')
        
        # forming params and value pairs
        if len(values) == 2:
            key, value = values
        elif len(values) == 3:
            key, value_1, value_2 = values
            value = value_1+":"+value_2
            if key == 'web address':
                catagory = value
        else:
            pass
        
        # put the value for each param of single website together
        data[catagory].setdefault(key,[]).append(value)
        #if key not in ['web address', 'Waterfall view']:
           # print data[catagory].setdefault(key)
      
        cleaned_dict = dict(data)
      

    for web in cleaned_dict.keys():
        count_num_data = count_num_data + 1
        #print count_num_data
        #print web
        del cleaned_dict[web]['Waterfall view']
        del cleaned_dict[web]['web address']

        for x in cleaned_dict[web].keys():
            if '' in cleaned_dict[web][x]:
                for y in cleaned_dict[web][x]:
                    cleaned_dict[web][x] = np.nan
        if cleaned_dict[web].has_key(''):
             del cleaned_dict[web]['']

        else:
            pass
    #print "The number of data collected is: ", count_nam_data

    return cleaned_dict

In [49]:
data=prepare(data)

In [50]:
def select(dataframe, param):
    dfT = dataframe.transpose()
    
    # covert back to frame
    df = pd.DataFrame(data=dfT[param]).dropna()
    
    # this link can be changed to .mean() for avarge
    # .min() for the minmun value
    # .max() for the max value
    s = df.apply(lambda x: pd.Series(x[param]), axis = 1).transpose().max()
    s.name = param
    
    # conver serias to DataFrame
    df_return = df.drop(param, axis=1).join(s)
    
    return df_return
    

In [51]:
df = pd.DataFrame(data=data)

In [52]:

def table(df):
    params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
 
    df_list = [select(df, param) for param in params]
    
    for indx, i in enumerate(df_list):
        
        # first time run
        if indx == 0:
            joined = i
        
        if indx < len(df_list) -1:
        
            joined = joined.join(df_list[indx+1])
            
    
       
    joined.index.names = ['Site name']

    return joined
#     df1 = pd.DataFrame(data=df_list[:1])
#     print df1
#     df2 = pd.DataFrame(data=df_list[1:2])
    #return pd.merge(*df_list, how='outer')
    

   

In [53]:
dfnew = table(df)
dfnew

,(Doc complete) Byets in,(Doc complete) Requests,(Fully loaded) Bytes in,(Fully loaded) Requests,(Fully loaded) Time,DOM elements,First byte,Load time,Speed Index,Start render
Site name,,,,,,,,,,
http://about.com/,4495425,38,4616725,68,10325,276,311,8239,2371,2100
http://amazon.co.jp/,4237284,201,5354939,306,13339,2295,1234,8920,6855,2700
http://aol.com/,1821899,108,1877615,151,12327,3252,1070,7218,4345,2100
http://apache.org/,574065,27,600165,33,2148,953,122,1828,962,900
http://bandcamp.com/,3826872,165,3828465,166,7354,4152,672,7294,2777,2000
http://bbb.org/,905540,66,916816,69,33297,600,750,32872,3441,3400
http://bbc.com/,1372121,101,1812136,255,20965,1360,418,5211,3194,2200
http://behance.net/,3379805,171,3385887,174,8419,2387,293,8249,2718,2400
http://beian.gov.cn/,1646040,42,1650991,43,12244,470,1547,11928,8731,7700


In [25]:
# dfnew.to_csv('./bokeh_app/data/formatetop200.csv')

## we can start making the model now

## Algorithm:

### Step1: Find min and max of each of the 7 parameters.

In [36]:
df_set = pd.read_csv('./bokeh_app/data/formatetop100.csv')

params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
# list 
param_values = {}
for param in params:
    param_values.update({
    param:[df_set[param].min(), df_set[param].mean(), df_set[param].max()],})


# for key in param_values:
#     print param_values[key]

df_param = pd.DataFrame.from_dict(param_values, orient='index')
df_param.columns=['Min','Average','Max']
#df_param
#df_param = pd.DataFrame(list(param_values.items()),columns=['Min','Average','Max'])


In [37]:
df_set.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
(Doc complete) Byets in,49,2283267.530612,2988632.278208,2999,527020,1159728,2853885,13786583
(Doc complete) Requests,49,79.020408,73.258529,5,24,61,85,369
(Fully loaded) Bytes in,49,2725607.244898,3601853.629268,3776,609950,1389638,3228037,17350231
(Fully loaded) Requests,49,109.795918,101.120180,6,39,71,130,408
(Fully loaded) Time,49,9204.122449,9720.931734,724,3191,4692,11162,43134
DOM elements,49,1123.020408,1113.947532,25,399,677,1421,6261
First byte,49,586.163265,277.521647,237,407,472,672,1359
Load time,49,6708.040816,7900.560014,577,2543,3759,7234,43065
Speed Index,49,3429.714286,2550.866487,600,1525,2675,4773,10769
Start render,49,1918.367347,1577.666192,600,1100,1500,2100,10700


### Step2: Normalize each parameter. After normalization, each parameter will have a value in between 0 and 1. 

If you want to normalize you data you can do as you suggest and simply calculate:
                 $$ zi=xi−min(x)/max(x)−min(x) $$
where x=(x1,...,xn) and zi is now your ith normalized data. As a proof of concept (although you did not ask for it) here is some R code and accompanying graph to illustrate this point:

In [38]:
def normalize(x, param, df_param):
    min_val = df_param.loc[param][0]
    max_val = df_param.loc[param][2]
    
    return (x - min_val)/(max_val - min_val)
    

In [39]:
normalize(700, 'Speed Index', df_param)

0.0098338086340839809

######  Step 3: iterate through each parameter and assign it points (from 1 to 100) on the basis of its normalized scale. For example, after normalization speed index is 0.2 for one website. Thinking intuitively, speed value should be more for good ranking and good performance. Now, we design our scale from 0 to 1 in such a way that high speed = high points. Example 1: calculating speed index

In [54]:
from sklearn import preprocessing

x = df_set.drop(['Site name'], axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns= ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',])
df.set_index(df_set['Site name'])

,(Doc complete) Byets in,(Doc complete) Requests,(Fully loaded) Bytes in,(Fully loaded) Requests,(Fully loaded) Time,DOM elements,First byte,Load time,Speed Index,Start render
Site name,,,,,,,,,,
http//addthis.com/,0.235381,0.173077,0.194598,0.179104,0.176256,0.148332,0.456328,0.159951,0.170125,0.118812
http//adobe.com/,0.351713,0.129121,0.282149,0.308458,0.285687,1.000000,0.278075,0.211471,0.993215,1.000000
http//amazon.com/,0.300577,0.510989,0.316968,0.793532,0.295591,0.445157,0.262032,0.173296,0.329629,0.069307
http//apple.com/,0.081110,0.118132,0.065714,0.106965,0.050200,0.081944,0.204991,0.051591,0.124791,0.069307
http//baidu.com/,0.015847,0.035714,0.013110,0.042289,0.093563,0.017960,0.283422,0.083977,0.118202,0.108911
http//bbc.co.uk/,0.211976,0.414835,0.186069,0.746269,0.561896,0.214240,0.781640,0.192078,0.410365,0.148515
http//bit.ly/,0.051566,0.041209,0.041053,0.039801,0.048314,0.071681,0.235294,0.044977,0.164028,0.148515
http//blogger.com/,0.100181,0.153846,0.079893,0.149254,0.064136,0.042816,0.265597,0.061170,0.379388,0.049505
http//blogspot.com/,0.100180,0.153846,0.079892,0.149254,0.065763,0.042816,0.349376,0.063006,0.379093,0.059406


--------------------------------------------------------------------------------------------------------------------------------
$$  If (speed > =0 and speed <= .1) {  $$  
$$  //assign lowest points to the speed parameter variable. Eg = assign 10 points $$ 
$$  If(speed >= 0.2 and speed =< .3){ $$ 
$$  //Assign 20 points $$ 
$$  If(speed > = .3 and speed =<.4 $$ 
$$  //Assign 30 points $$ 
$$  And so on – for speed index - the points we assign increases as the value of the parameter lies in $$ 
$$  the range closer to 1. $$ 

Example 2: calculating load time
If (load time > =0 and speed <= .1) {

//assign highest points to the speed parameter variable. Eg = assign 90 points
If(speed >= 0.1 and speed =< .2){
// Assign 80 points
If(speed > = .2 and speed =<.3)
//Assign 70 points
And so on .. for load time - the points decreases as the value of the parameter lying in the range
is closer to 0.
------------------------------------------------------------------------------------------------------------------------------------------

Likewise, we do this step for each parameter for each website.
At the end of this step, we get total optimal points for each website having: speed points + load
time points + ...all other parameter points.

Step 4:
Now, we have the total points depending on the scale of each of the 100 websites calculated.

Step 5:
Compare the optimal points calculated now to get the ranking. As we have normalized the scale,
and have the model having total optimal points for each website.

Step 6:
Sort the total points of each website and get the ranking info depending on the sort. One with
the maximum points calculated is the ranked first and so on..

Step 7 : Once we have the ranking, if user wants to improve the ranking of a given website, so
basically increase the points for that website.

//Calculate increase of points required to make a website No.1
Points_rank1_website = p1;
Current_ Points_of_website_user_wants_to_improve = p2;
Points_needed = (points of rank 1 website – point of website user wants to improve to rank 1)
//Then increase those many points

Points_of_website_user_wants_to_improve = Points_of_website_user_wants_to_improve +
points_needed;

Step 7:
Depending on the new points, again sort the list and display ranking. Now user has the provision
to improve the ranking of any website and display it.